In [1]:
import lzma
import shutil
import tempfile
from pathlib import Path

import geopandas as gpd
import pandas as pd

In [2]:
INPUT_DATA_DIR = Path.cwd() / ".." / ".." / ".." / "data" / "shapefiles" / "input"
OUTPUT_DATA_DIR = Path.cwd() / ".." / ".." / ".." / "data" / "shapefiles" / "output"

INPUT_DATA_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DATA_DIR.mkdir(parents=True, exist_ok=True)

RI_EPSG = 3438  # State plane; ft

In [3]:
# Data from rigis.org
# https://www.rigis.org/datasets/urban-areas/explore
# https://www.rigis.org/datasets/municipalities-1997/explore

with tempfile.TemporaryDirectory() as tmpdir:
    tmpdir = Path(tmpdir)
    with lzma.open(INPUT_DATA_DIR / "Urban_Areas.geojson.xz", 'rt') as infile:
        with open(tmpdir / "Urban_Areas.geojson", 'wt') as outfile:
            shutil.copyfileobj(infile, outfile)
    urban_gdf = gpd.read_file(tmpdir / "Urban_Areas.geojson").to_crs(epsg=RI_EPSG)
    
    with lzma.open(INPUT_DATA_DIR / "Municipalities_(1997).geojson.xz", 'rt') as infile:
        with open(tmpdir / "Municipalities_(1997).geojson", 'wt') as outfile:
            shutil.copyfileobj(infile, outfile)
    
    muni_gdf = gpd.read_file(tmpdir / "Municipalities_(1997).geojson").to_crs(epsg=RI_EPSG)

In [4]:
urban_gdf = urban_gdf.dissolve()
muni_gdf = muni_gdf.dissolve(['NAME', 'COUNTY']).reset_index()

In [5]:
joined = gpd.sjoin(
    urban_gdf[['geometry']],
    muni_gdf[['NAME', 'COUNTY', 'geometry']],
).reset_index(drop=True)

joined['muni_geometry'] = muni_gdf.loc[joined['index_right'], 'geometry'].reset_index(drop=True)

joined['intersection_geometry'] = joined['geometry'].intersection(joined['muni_geometry'])

In [6]:
joined['percent_urban'] = joined['intersection_geometry'].area / joined['muni_geometry'].area

In [7]:
final_df = pd.concat([
    joined[['NAME', 'COUNTY', 'percent_urban']],
    muni_gdf[~muni_gdf['NAME'].isin(joined['NAME'])][['NAME', 'COUNTY']]
]).fillna(0)

In [8]:
final_df.to_csv(OUTPUT_DATA_DIR / 'urban_percent.csv', index=False)